Downloading Data:
=================

- Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
- Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5

Links:
- https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process/Examples/S2L2A.html#true-color
- https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/cloudless_process_api.html



In [1]:
# Print basic timestamp and python version information

import time
import sys

print("Timestamp: ", time.strftime("%Y-%m-%d %H:%M:%S"))
print("Python version: ", sys.version)

Timestamp:  2025-01-27 15:44:10
Python version:  3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [41]:
# Setup the environment with pip installs and imports

installs = ["sentinelhub", "numpy", "pandas", "matplotlib", "requests", "getpass", "scipy"]
for package in installs:
    !pip install {package}

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/43.6 MB ? eta -:--:--
    --------------------------------------- 1.0/43.6 MB 3.3 MB/s eta 0:00:13
   -- ------------------------------------- 2.4/43.6 MB 5.0 MB/s eta 0:00:09
   --- ------------------------------------ 3.9/43.6 MB 5.7 MB/s eta 0:00:07
   ----- ---------------------------------- 5.8/43.6 MB 6.5 MB/s eta 0:00:06
   ------ --------------------------------- 7.6/43.6 MB 7.1 MB/s eta 0:00:06
   -------- ------------------------------- 9.4/43.6 MB 7.2 MB/s eta 0:00:05
   ---------- ----------------------------- 11.3/43.6 MB 7.6 MB/s eta 0:00:05
   ------------ --------------------------- 13.1/43.6 MB 7.8 MB/s eta 0:00:04
   ------------- -------------------------- 14.9/43.6 MB 7.9 MB/s eta 0:00:04
   --------------- ------------------------ 17.0/43.6 MB 8.1 MB/s eta 0:00:04
   ----------------- ---------------------- 18.9/43.6 MB 8.2 MB/s eta 0:00:04
   -----

In [2]:


# Utilities
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)


In [3]:
# Your client credentials
#Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
#Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5
client_id = 'sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9'
client_secret = 'BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5'

In [4]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

def SetupOAuth(client_id, client_secret):

    # Create a session
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)

    # Get token for the session
    token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                            client_secret=client_secret, include_client_id=True)

    # All requests using this session will have an access token automatically added
    resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")

    return oauth

oauth = SetupOAuth(client_id, client_secret)


In [6]:
# Create strings to support the query

def CreateBBoxString(minlon, minlat, maxlon, maxlat):
    return f"[{minlon}, {minlat}, {maxlon}, {maxlat}]"

def CreateTimerangeString(date):
    # add buffer around the stated date
    datespace = datetime.timedelta(days=2)
    startdate = date - datespace
    enddate = date + datespace

    # format individual date strings
    from_date = f"{startdate.strftime('%Y-%m-%d')}T00:00:00Z"
    to_date = f"{enddate.strftime('%Y-%m-%d')}T23:59:59Z"

    # return the collated/formatted answer
    return {"from": from_date, "to": to_date}



In [7]:
from PIL import Image
from io import BytesIO


def download_GM(outpath, bl_lat, bl_lon, width):

    minlat = bl_lat
    maxlat = bl_lat + width
    minlon = bl_lon
    maxlon = bl_lon + width

    evalscript_GM = """
    //VERSION=3
    function setup() {
        return {
            input: ["B02", "B03", "B04"],
            output: { bands: 3 }
        };
    }

    function evaluatePixel(sample) {
        return [3.5 * sample.B04/10000, 3.5 * sample.B03/10000, 3.5 * sample.B02/10000];
    }
    """

    request_GM = {
    "input": {
        "bounds": {
        "bbox": [minlon, minlat, maxlon, maxlat]
        },
        "data": [
        {
            "dataFilter": {
            "timeRange": {
                "from": "2023-07-01T00:00:00Z",
                "to": "2023-07-01T23:59:59Z"
            }
            },
            "type": "byoc-5460de54-082e-473a-b6ea-d5cbe3c17cca"

        }
        ]
    },
    "output": {
        "width": 1024,
        "height": 1024,
        "responses": [
        {
            "identifier": "default",
            "format": {
                "type": "image/png"
            }
        }
        ]
    },
    "evalscript": evalscript_GM,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request_GM)

    # Output the result to a file
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(outpath)
    else:
        print(f"Request failed with status code {response.status_code}")



In [8]:
import os

# function to make sure a path exists. If not, create it
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [10]:
# Data structure for the metadata
image_data_5Deg = [
    {"filename": "Sat_EL_DD.png", "lat": -50, "long": 165, "width": 5},
    {"filename": "Sat_EL_CD.png", "lat": -45, "long": 165, "width": 5},
    {"filename": "Sat_EL_DE.png", "lat": -50, "long": 170, "width": 5},
]

image_data_1Deg = [
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_CC.png", "lat": -48, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_CD.png", "lat": -48, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BB.png", "lat": -47, "long": 166, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BC.png", "lat": -47, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BD.png", "lat": -47, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BE.png", "lat": -47, "long": 169, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AB.png", "lat": -46, "long": 166, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AC.png", "lat": -46, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AD.png", "lat": -46, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AE.png", "lat": -46, "long": 169, "width": 1},

    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_EC.png", "lat": -45, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_ED.png", "lat": -45, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_EE.png", "lat": -45, "long": 169, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DC.png", "lat": -44, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DD.png", "lat": -44, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DE.png", "lat": -44, "long": 169, "width": 1},

    {"localpath": "Lvl2_1x1/EL_DE/", "filename": "Sat_EL_DE_AA.png", "lat": -46, "long": 170, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DE/", "filename": "Sat_EL_DE_BA.png", "lat": -47, "long": 170, "width": 1},
]


iceland_data_1Deg = [
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BC.png", "lat": 63, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BD.png", "lat": 63, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BE.png", "lat": 63, "long": -21, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AA.png", "lat": 64, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AB.png", "lat": 64, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AC.png", "lat": 64, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AD.png", "lat": 64, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AE.png", "lat": 64, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EA.png", "lat": 65, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EB.png", "lat": 65, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EC.png", "lat": 65, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_ED.png", "lat": 65, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EE.png", "lat": 65, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DA.png", "lat": 66, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DB.png", "lat": 66, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DC.png", "lat": 66, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DD.png", "lat": 66, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DE.png", "lat": 66, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AA.png", "lat": 64, "long": -20, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AB.png", "lat": 64, "long": -19, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AC.png", "lat": 64, "long": -18, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AD.png", "lat": 64, "long": -17, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AE.png", "lat": 64, "long": -16, "width": 1},

    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BA.png", "lat": 63, "long": -20, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BB.png", "lat": 63, "long": -19, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BC.png", "lat": 63, "long": -18, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BD.png", "lat": 63, "long": -17, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BE.png", "lat": 63, "long": -16, "width": 1},


]

In [11]:
australia_data_5Deg = [
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_DE.png", "lat": -20, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_DF.png", "lat": -20, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_EE.png", "lat": -25, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_EF.png", "lat": -25, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_FE.png", "lat": -30, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_FF.png", "lat": -30, "long": 115, "width": 5},


    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BA.png", "lat": -10, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BB.png", "lat": -10, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BC.png", "lat": -10, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BD.png", "lat": -10, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BE.png", "lat": -10, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BF.png", "lat": -10, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CA.png", "lat": -15, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CB.png", "lat": -15, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CC.png", "lat": -15, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CD.png", "lat": -15, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CE.png", "lat": -15, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CF.png", "lat": -15, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DA.png", "lat": -20, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DB.png", "lat": -20, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DC.png", "lat": -20, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DD.png", "lat": -20, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DE.png", "lat": -20, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DF.png", "lat": -20, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EA.png", "lat": -25, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EB.png", "lat": -25, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EC.png", "lat": -25, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_ED.png", "lat": -25, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EE.png", "lat": -25, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EF.png", "lat": -25, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FA.png", "lat": -30, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FB.png", "lat": -30, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FC.png", "lat": -30, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FD.png", "lat": -30, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FE.png", "lat": -30, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FF.png", "lat": -30, "long": 145, "width": 5},


    {"localpath": "Lvl1_5x5/DL/", "filename": "Sat_DL_EA.png", "lat": -25, "long": 150, "width": 5},
    {"localpath": "Lvl1_5x5/DL/", "filename": "Sat_DL_FA.png", "lat": -30, "long": 150, "width": 5},

    {"localpath": "Lvl1_5x5/EJ/", "filename": "Sat_EJ_AE.png", "lat": -35, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/EJ/", "filename": "Sat_EJ_AF.png", "lat": -35, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AA.png", "lat": -35, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AB.png", "lat": -35, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AC.png", "lat": -35, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AD.png", "lat": -35, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AE.png", "lat": -35, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AF.png", "lat": -35, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BD.png", "lat": -40, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BE.png", "lat": -40, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BF.png", "lat": -40, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_CE.png", "lat": -45, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_CF.png", "lat": -45, "long": 145, "width": 5},


    {"localpath": "Lvl1_5x5/EL/", "filename": "Sat_EL_AA.png", "lat": -35, "long": 150, "width": 5},
    {"localpath": "Lvl1_5x5/EL/", "filename": "Sat_EL_BA.png", "lat": -40, "long": 150, "width": 5},


]

In [12]:

iberia_data_5Deg = [
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_DE.png", "lat": 40, "long": -10, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_DF.png", "lat": 40, "long":  -5, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_EE.png", "lat": 35, "long": -10, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_EF.png", "lat": 35, "long":  -5, "width": 5},
]

iberia_data_1Deg = [
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AA.png", "lat": 44, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AB.png", "lat": 44, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AC.png", "lat": 44, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AD.png", "lat": 44, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AE.png", "lat": 44, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BA.png", "lat": 43, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BB.png", "lat": 43, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BC.png", "lat": 43, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BD.png", "lat": 43, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BE.png", "lat": 43, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CA.png", "lat": 42, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CB.png", "lat": 42, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CC.png", "lat": 42, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CD.png", "lat": 42, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CE.png", "lat": 42, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DA.png", "lat": 41, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DB.png", "lat": 41, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DC.png", "lat": 41, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DD.png", "lat": 41, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DE.png", "lat": 41, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EA.png", "lat": 40, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EB.png", "lat": 40, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EC.png", "lat": 40, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_ED.png", "lat": 40, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EE.png", "lat": 40, "long":  -6, "width": 1},



    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AA.png", "lat": 44, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AB.png", "lat": 44, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AC.png", "lat": 44, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AD.png", "lat": 44, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AE.png", "lat": 44, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BA.png", "lat": 43, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BB.png", "lat": 43, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BC.png", "lat": 43, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BD.png", "lat": 43, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BE.png", "lat": 43, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CA.png", "lat": 42, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CB.png", "lat": 42, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CC.png", "lat": 42, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CD.png", "lat": 42, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CE.png", "lat": 42, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DA.png", "lat": 41, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DB.png", "lat": 41, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DC.png", "lat": 41, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DD.png", "lat": 41, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DE.png", "lat": 41, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_EA.png", "lat": 40, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_EB.png", "lat": 40, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_EC.png", "lat": 40, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_ED.png", "lat": 40, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_EE.png", "lat": 40, "long":  -1, "width": 1},



    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AA.png", "lat": 39, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AB.png", "lat": 39, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AC.png", "lat": 39, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AD.png", "lat": 39, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AE.png", "lat": 39, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BA.png", "lat": 38, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BB.png", "lat": 38, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BC.png", "lat": 38, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BD.png", "lat": 38, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BE.png", "lat": 38, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CA.png", "lat": 37, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CB.png", "lat": 37, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CC.png", "lat": 37, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CD.png", "lat": 37, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CE.png", "lat": 37, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DA.png", "lat": 36, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DB.png", "lat": 36, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DC.png", "lat": 36, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DD.png", "lat": 36, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DE.png", "lat": 36, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_EA.png", "lat": 35, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_EB.png", "lat": 35, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_EC.png", "lat": 35, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_ED.png", "lat": 35, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_EE.png", "lat": 35, "long":  -6, "width": 1},



    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AA.png", "lat": 39, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AB.png", "lat": 39, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AC.png", "lat": 39, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AD.png", "lat": 39, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AE.png", "lat": 39, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BA.png", "lat": 38, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BB.png", "lat": 38, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BC.png", "lat": 38, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BD.png", "lat": 38, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BE.png", "lat": 38, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CA.png", "lat": 37, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CB.png", "lat": 37, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CC.png", "lat": 37, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CD.png", "lat": 37, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CE.png", "lat": 37, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DA.png", "lat": 36, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DB.png", "lat": 36, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DC.png", "lat": 36, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DD.png", "lat": 36, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DE.png", "lat": 36, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_EA.png", "lat": 35, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_EB.png", "lat": 35, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_EC.png", "lat": 35, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_ED.png", "lat": 35, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_EE.png", "lat": 35, "long":  -1, "width": 1},

]

In [15]:
westmed_data_5Deg = [
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DA.png", "lat": 40, "long":   0, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DB.png", "lat": 40, "long":   5, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_EA.png", "lat": 35, "long":   0, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_EB.png", "lat": 35, "long":   5, "width": 5},
]

westmed_data_1Deg = [
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AA.png", "lat": 44, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AB.png", "lat": 44, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AC.png", "lat": 44, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AD.png", "lat": 44, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AE.png", "lat": 44, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BA.png", "lat": 43, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BB.png", "lat": 43, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BC.png", "lat": 43, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BD.png", "lat": 43, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BE.png", "lat": 43, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CA.png", "lat": 42, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CB.png", "lat": 42, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CC.png", "lat": 42, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CD.png", "lat": 42, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CE.png", "lat": 42, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DA.png", "lat": 41, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DB.png", "lat": 41, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DC.png", "lat": 41, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DD.png", "lat": 41, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DE.png", "lat": 41, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_EA.png", "lat": 40, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_EB.png", "lat": 40, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_EC.png", "lat": 40, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_ED.png", "lat": 40, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_EE.png", "lat": 40, "long": 4, "width": 1},

]


In [24]:
france_data_5Deg = [
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_CF.png", "lat": 45, "long": -5, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_CA.png", "lat": 45, "long":  0, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_CB.png", "lat": 45, "long":  5, "width": 5},
]


france_data_1Deg = [
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AA.png", "lat": 49, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AB.png", "lat": 49, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AC.png", "lat": 49, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AD.png", "lat": 49, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AE.png", "lat": 49, "long": -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BA.png", "lat": 48, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BB.png", "lat": 48, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BC.png", "lat": 48, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BD.png", "lat": 48, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BE.png", "lat": 48, "long": -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CA.png", "lat": 47, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CB.png", "lat": 47, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CC.png", "lat": 47, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CD.png", "lat": 47, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CE.png", "lat": 47, "long": -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DA.png", "lat": 46, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DB.png", "lat": 46, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DC.png", "lat": 46, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DD.png", "lat": 46, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DE.png", "lat": 46, "long": -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_EA.png", "lat": 45, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_EB.png", "lat": 45, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_EC.png", "lat": 45, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_ED.png", "lat": 45, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_EE.png", "lat": 45, "long": -1, "width": 1},



    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AA.png", "lat": 49, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AB.png", "lat": 49, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AC.png", "lat": 49, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AD.png", "lat": 49, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AE.png", "lat": 49, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BA.png", "lat": 48, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BB.png", "lat": 48, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BC.png", "lat": 48, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BD.png", "lat": 48, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BE.png", "lat": 48, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CA.png", "lat": 47, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CB.png", "lat": 47, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CC.png", "lat": 47, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CD.png", "lat": 47, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CE.png", "lat": 47, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DA.png", "lat": 46, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DB.png", "lat": 46, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DC.png", "lat": 46, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DD.png", "lat": 46, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DE.png", "lat": 46, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_EA.png", "lat": 45, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_EB.png", "lat": 45, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_EC.png", "lat": 45, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_ED.png", "lat": 45, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_EE.png", "lat": 45, "long": 4, "width": 1},


]

In [22]:
belgium_data_5Deg = [
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_BA.png", "lat": 50, "long": 0, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_BB.png", "lat": 50, "long": 5, "width": 5},
]


In [25]:
RootDir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/"
ensure_dir(RootDir)

brightness = 35

oauth = SetupOAuth(client_id, client_secret)


# Loop over the data structure
for data in france_data_5Deg:

    TilesetDir = os.path.join(RootDir, data["localpath"])
    filepath   = os.path.join(TilesetDir, data["filename"])

    ensure_dir(TilesetDir)

    print(filepath)

    if not os.path.exists(filepath):

        print(f"Downloading and enhancing image: {data['filename']}")

        # Call your functions using the metadata
        download_GM(filepath, data["lat"], data["long"], data["width"])
        # change_absolute_brightness(filepath, filepath, brightness)
        #adjust_image_properties(filepath, filepath, brightness_delta=30, contrast_factor=1.5, saturation_factor=1.5)





C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl1_5x5/BF/Sat_BF_CF.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl1_5x5/BG/Sat_BG_CA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl1_5x5/BG/Sat_BG_CB.png
